In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import config
os.chdir(config.LOOP_CATALOG_DIR)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

## Helper Functions

In [63]:
def read_out_log(log):
    
    status = 0
    sample_name = 'check'
    paired = 'check'

    # extract job_id and index
    meta = os.path.basename(log)
    job_id = meta.split('.')[1]
    #job_id = meta[0].replace('o', '')
    #index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'Running' in info and "16G" in info:
                sample_name = info.split("-n")[1].split(" -g")[0]
            if 'This sample used' in info:
                paired = info.split()[3]
            if 'Ended:' in info:
                status = 1

    #return([index, sample_name, read_length, job_id, status])
    return([sample_name, paired, job_id, status])

In [64]:
def read_error_log(log):
    
    status = 1
    
    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'error' in info:
                status = 0
            if 'Error' in info:
                status = 0
    
    return(status)

## Check Logs

In [65]:
# create a dataframe of log information
output_logs = glob.glob('results/peaks/chipline_v2/logs/run_chipline/run_chipline*.out')
log_data = []   
for out_log in output_logs:
    sample_index = out_log.split('.')[1]
    #index, sample_name, read_length, job_id, out_status = read_out_log(out_log)
    #print(out_log)
    sample_name, paired, job_id, out_status = read_out_log(out_log)
    error_log = [log for log in glob.glob('results/peaks/chipline_v2/logs/run_chipline/run_chipline*.err') if log.split("/")[5].split('.')[1] == sample_index][0]
    error_status = read_error_log(error_log)
    #log_data.append([index, sample_name, read_length, job_id, out_status, error_status, os.path.basename(out_log)])
    log_data.append([sample_name, paired, job_id, out_status, error_status, os.path.basename(out_log)])
log_df = pd.DataFrame(log_data)
#log_df.columns = ['sample_index', 'std_sample_name', 'read_length', 'job_id', 'out_status', 'error_status', 'log']
log_df.columns = ['std_sample_name', 'paired', 'job_id', 'out_status', 'error_status', 'log']

In [66]:
log_df = log_df.sort_values(by=["std_sample_name"], ascending=True).reset_index(drop=True)
log_df

,std_sample_name,paired,job_id,out_status,error_status,log
0,BMDC-CTCF_cKO-LPS.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8805532,1,1,run_chipline.8805532.8.out
1,BMDC-CTCF_cKO.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8805532,1,1,run_chipline.8805532.7.out
2,BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8816730,1,1,run_chipline.8816730.3.out
3,BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8816730,1,1,run_chipline.8816730.2.out
4,BMDC-LPS.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8816730,1,1,run_chipline.8816730.4.out
5,BMDC-WT-LPS.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8805532,1,1,run_chipline.8805532.13.out
6,BMDC-WT.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8805532,1,1,run_chipline.8805532.12.out
7,BMDC.GSE185880.Mus_Musculus.H3K27ac.b1,paired-end,8816729,1,1,run_chipline.8816729.1.out
8,CD4N_merged_donors.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,single-end,8815742,1,1,run_chipline.8815742.1.out
9,CD8N_merged_donors.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,single-end,8815743,1,1,run_chipline.8815743.2.out


In [13]:
" ".join(list(log_df["std_sample_name"]))

'Bergeron-697-0hr.GSE188713.Homo_Sapiens.H3K27ac.b1 Bergeron-697-24hr.GSE188713.Homo_Sapiens.H3K27ac.b1 Bergeron-697-6hr.GSE188713.Homo_Sapiens.H3K27ac.b1 HCT116-FOXD2.GSE196920.Homo_Sapiens.H3K27ac.b1 HCT116-FOXD2.GSE196920.Homo_Sapiens.H3K27ac.b2 HCT116-FOXD2.GSE196920.Homo_Sapiens.H3K4me1.b1 HCT116-FOXD2.GSE196920.Homo_Sapiens.H3K4me1.b2 HCT116-FOXD2.GSE196920.Homo_Sapiens.H3K4me1.b3 HCT116-FOXD2.GSE196920.Homo_Sapiens.H3K4me3.b1 HCT116-FOXD2.GSE196920.Homo_Sapiens.H3K4me3.b2 HCT116.GSE196920.Homo_Sapiens.H3K27ac.b1 HCT116.GSE196920.Homo_Sapiens.H3K27ac.b2 HCT116.GSE196920.Homo_Sapiens.H3K4me1.b1 HCT116.GSE196920.Homo_Sapiens.H3K4me1.b2 HCT116.GSE196920.Homo_Sapiens.H3K4me1.b3 HCT116.GSE196920.Homo_Sapiens.H3K4me3.b1 HCT116.GSE196920.Homo_Sapiens.H3K4me3.b2 Nalm6-0hr.GSE188713.Homo_Sapiens.H3K27ac.b1 Nalm6-24hr.GSE188713.Homo_Sapiens.H3K27ac.b1 Nalm6-6hr.GSE188713.Homo_Sapiens.H3K27ac.b1 RD.GSE245495.Homo_Sapiens.H3K27ac.b1 RH4.GSE245495.Homo_Sapiens.H3K27ac.b1 T47D-T0.GSE179666.Hom

## Identifying problem samples

In [9]:
problems = (log_df.out_status != 0) | (log_df.error_status != 0)
problems_df = log_df.loc[problems,:]
print('nubmer of problem samples:', len(problems_df))

nubmer of problem samples: 0


In [10]:
problems_df.loc[:, ['std_sample_name', 'job_id', 'out_status', 'error_status', 'log']]

,std_sample_name,job_id,out_status,error_status,log


In [24]:
" ".join(list(problems_df["std_sample_name"]))

'RD.GSE245495.Homo_Sapiens.H3K27ac.b1 T47D-T0.GSE179666.Homo_Sapiens.PGR.b1 T47D-T30.GSE179666.Homo_Sapiens.PGR.b1 VCaP-AU-4h.GSE171591.Homo_Sapiens.H3K4me3.b1 YAP-M-tumor.GSE207801.Mus_Musculus.H3K27ac.b2 mNSC.GSE207801.Mus_Musculus.H3K27ac.b2'

In [42]:
ids = []
ss = "results/samplesheets/hicpro/current.hicpro.samplesheet.merged.batch2.without_header.tsv"
samples = list(log_df.loc[log_df["std_sample_name"].str.contains("phs")]["std_sample_name"])
count = 1
with open(ss) as s:
    for line in s:
        if line.strip().split("\t")[0] in samples:
            ids.append(str(count))
        count += 1
print(",".join(ids))

21,22,23,96,97,98


In [44]:
samples

['CD4_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.biorep_merged',
 'CD4_Naive_1829-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.biorep_merged',
 'CD4_Naive_1831-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.biorep_merged',
 'Monocyte_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.biorep_merged',
 'Monocyte_1814-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.biorep_merged',
 'Monocyte_1816-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.biorep_merged']

In [17]:
header_output = '{}.with_header.tsv'.format(output_prefix)
problems_df.to_csv(header_output, header=True, index=False, sep='\t')